## **사용자 정의 함수X**

In [1]:
import sklearn.base as skbase
import sklearn.pipeline as skpi
import sklearn.impute as skim
import sklearn.preprocessing as skpre
import sklearn.compose as skcom

In [2]:
import pandas as pd
df = pd.read_excel(r'C:\Users\hanjieun_2\Desktop\개인공부\algorithms\1.Supervised_Learning\1.Classification\6.Support_Vector_Machine\Raisin_Dataset.xlsx')
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             900 non-null    int64  
 1   MajorAxisLength  900 non-null    float64
 2   MinorAxisLength  900 non-null    float64
 3   Eccentricity     900 non-null    float64
 4   ConvexArea       900 non-null    int64  
 5   Extent           900 non-null    float64
 6   Perimeter        900 non-null    float64
 7   Class            900 non-null    object 
dtypes: float64(5), int64(2), object(1)
memory usage: 56.4+ KB


### 1. 컬럼 분리

In [5]:
import numpy as np
target = df['Class']
num_cols = df.select_dtypes(np.number).columns.difference(['Class'])
object_cols = df.select_dtypes('object').columns

### 2. 전처리 과정 세분화
#### (1) 연속형 변수

In [6]:
num_pipe = skpi.make_pipeline(
    # 결측치
    skim.SimpleImputer(strategy='mean'), 
    # Scaling
    skpre.StandardScaler()
)

In [7]:
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

#### (2) 범주형 변수

In [8]:
object_pipe = skpi.make_pipeline(
    # 결측치
    skim.SimpleImputer(strategy='most_frequent'),
    # onehotEncoding
    skpre.OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)
)

- handle_unknown='ignore':
이 파라미터는 범주형 변수를 인코딩하는 동안 알려지지 않은 범주가 나타날 때의 처리 방법을 지정합니다.
'ignore'로 설정되면 알려지지 않은 범주가 나타날 경우 무시하고 처리합니다.
예를 들어, 범주형 변수에 대한 원-핫 인코딩을 수행할 때, 새로운 범주가 발생하면 무시하고 해당 열을 추가하지 않습니다.

- drop='first':
이 파라미터는 다중공선성을 줄이기 위해 더미 변수 중 하나를 삭제할지 여부를 지정합니다.
'first'로 설정되면 첫 번째 더미 변수가 삭제됩니다.
다중공선성은 독립 변수 간의 상관 관계가 높아 예측 모델에 불안정성을 초래할 수 있는 문제입니다. 더미 변수를 생성할 때 첫 번째 변수를 삭제하여 이러한 문제를 방지할 수 있습니다.

- sparse_output=False:
이 파라미터는 결과를 희소 행렬로 반환할지 여부를 지정합니다.
False로 설정되면 결과를 밀집 행렬로 반환합니다. 즉, 0이 아닌 모든 값을 포함하는 행렬입니다.
희소 행렬은 대부분의 요소가 0인 행렬을 나타내며, 메모리 효율적으로 저장될 수 있습니다. 특히, 원-핫 인코딩과 같이 대부분의 값이 0인 경우에 유용합니다.

In [9]:
object_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(drop='first', handle_unknown='ignore',
                               sparse_output=False))])

### (3) 합치기

In [10]:
total_pipe = skcom.make_column_transformer(
    (num_pipe, num_cols),
    (object_pipe, object_cols),
    remainder='passthrough'
)

In [11]:
total_pipe

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Area', 'ConvexArea', 'Eccentricity', 'Extent', 'MajorAxisLength',
       'MinorAxisLength', 'Perimeter'],
      dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Class'], dtype='object'))])

In [12]:
total_pipe.fit(df.head(5000))

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Area', 'ConvexArea', 'Eccentricity', 'Extent', 'MajorAxisLength',
       'MinorAxisLength', 'Perimeter'],
      dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Class'], dtype='object'))])

In [13]:
total_pipe.transform(df.head(5000))

array([[-0.00718637, -0.01570903,  0.42314164, ..., -0.02395819,
         0.06627397,  1.        ],
       [-0.32421719, -0.3042482 ,  0.22447639, ..., -0.22929222,
        -0.16125245,  1.        ],
       [ 0.07829241,  0.06211335,  0.18623907, ...,  0.23698797,
         0.15594468,  1.        ],
       ...,
       [ 0.30407232,  0.37004063, -0.6631136 , ...,  0.88767401,
         0.46387322,  0.        ],
       [ 0.14671134,  0.15871029,  0.71117344, ..., -0.00624644,
         0.3385864 ,  0.        ],
       [-0.05631357, -0.04881606,  1.39366043, ..., -0.78493347,
         0.39090133,  0.        ]])

### (4) 컬럼 이름 설정해주기

In [14]:
total_pipe.get_feature_names_out()

array(['pipeline-1__Area', 'pipeline-1__ConvexArea',
       'pipeline-1__Eccentricity', 'pipeline-1__Extent',
       'pipeline-1__MajorAxisLength', 'pipeline-1__MinorAxisLength',
       'pipeline-1__Perimeter', 'pipeline-2__Class_Kecimen'], dtype=object)

In [16]:
def columns_name(x):
    x = x.replace('pipeline-1__', '')
    x = x.replace('pipeline-2__', '')
    x = x.replace('remainder__', '')
    return x

In [17]:
col_names = list(map(columns_name, total_pipe.get_feature_names_out()))

In [18]:
data = pd.DataFrame(total_pipe.transform(df.head(5000)), columns=col_names)
data.head()

,Area,ConvexArea,Eccentricity,Extent,MajorAxisLength,MinorAxisLength,Perimeter,Class_Kecimen
0,-0.007186,-0.015709,0.423142,1.106743,0.097577,-0.023958,0.066274,1.0
1,-0.324217,-0.304248,0.224476,-0.287777,-0.209012,-0.229292,-0.161252,1.0
2,0.078292,0.062113,0.186239,-1.158250,0.097758,0.236988,0.155945,1.0
3,-1.074286,-1.076165,-1.069623,0.001711,-1.245051,-0.915273,-1.175915,1.0
4,-0.215393,-0.238623,-2.409827,1.745259,-0.678958,0.727354,-0.338639,1.0


************

## **사용자 정의 함수O (Class사용)** 

In [1]:
import sklearn.base as base
import sklearn.preprocessing as skpre

import sklearn.pipeline as skpi
import sklearn.impute as skim
import sklearn.compose as skcom

In [3]:
class Passthrough(base.BaseEstimator, base.TransformerMixin):
    # 기본 함수 정의
    def __init__(self) -> None:
        super().__init__()
        self.column_name = None
    # 학습 함수 + 파이프라인은 칼럼이름 기억 못하기 떄문에 저장
    def fit(self, x):
        self.column_name = list(x.columns)
        return self
    # 저장해놓은 칼럼이름 호출
    def get_feature_names_out(self, x=None):
        return self.column_name
    # 변환
    def transform(self, x):
        return x.values

In [ ]:
# 종속변수

# 독립변수
# 연속형 변수

# 범주형 변수

# 파이프라인 합치기
# 연속형 + 범주형 + remainder